<a href="https://colab.research.google.com/github/tokyo8182/LLM-RAG/blob/main/Llamaparse_%2B_groq_pipeline_for_metadata_for_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Read in all files i.e. data ingestion

Basic csv reading code added. Next: use llamaparse's own data loaders

In [1]:
# connecting to google drive drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os

# The CSV download link
path = '/content/drive/MyDrive/Omdena_Challenge/new_LK_tea_dataset/new_LK_tea_dataset_updated.csv'

# Read the Google Sheet into a pandas DataFrame
all_data = pd.read_csv(path , index_col=0)
all_data.fillna('', inplace=True)
# Display the DataFrame
all_data.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Omdena_Challenge/new_LK_tea_dataset/new_LK_tea_dataset_updated.csv'

In [ ]:
import pandas as pd

# Function to count words and characters in text
def count_words_and_characters(text):
    words = text.split()  # Split by whitespace to get words
    num_words = len(words)
    num_characters = len(text)
    return num_words, num_characters

# Function to count words and characters for the 'text_content' column in the DataFrame
def count_stats_in_dataframe(df):
    total_words = 0
    total_characters = 0

    # Iterate over the 'text_content' column in the DataFrame
    for text in df['text_content'].dropna():  # Drop any NaN values in text_content
        num_words, num_characters = count_words_and_characters(text)
        total_words += num_words
        total_characters += num_characters

    return total_words, total_characters

# Main script
if __name__ == "__main__":

    # Count words and characters
    total_words, total_characters = count_stats_in_dataframe(all_data)

    # Output the results
    print("\nTotal Stats for 'text_content' column in DataFrame:")
    print(f"Total Words: {total_words}")
    print(f"Total Characters: {total_characters}")



Total Stats for 'text_content' column in DataFrame:
Total Words: 186342
Total Characters: 1120744


In [ ]:
# # for th future, can use paths to check where path ends in .pdf
# # Filter rows where the 'path' column ends with '.pdf'

# data_pdf = all_data[all_data['PDF_or_text'] == 'PDF']

In [ ]:
# # modify to add text_only in either path or new column
# data_text = all_data[all_data['PDF_or_text'] == 'Text']


# 2. Feed all the PDF files as well as ACTs to llamaparse and get parsed chunks in markdown format

for 173 files, around 600 chunks are returned

@TODO convert ACTs via Fast mode (1 cred. / 3 p)
@TODO convert PDFs via Accurate mode (1 cred. / 1 p.), and don't use continuous mode (10 credits)...except for PDFs with Tables....becuase if they span multiple pages, it would be an issue.....................

In [ ]:
!pip --quiet install llama-index-core llama-parse llama-index-readers-file python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
import getpass
os.environ['LLAMA_CLOUD_API_KEY'] = getpass.getpass('Enter your LLamacloud API Key: ')
# os.environ['GROQ_API_KEY'] = getpass.getpass('Enter your GROQ API Key: ')

Enter your LLamacloud API Key: ··········


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# my version



import pandas as pd
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# Load environment variables (assuming LLAMA_CLOUD_API_KEY is needed somewhere)
load_dotenv()

# Assuming you have your DataFrame `all_data` already loaded
# Example: all_data = pd.read_csv("your_data.csv") or whatever source you're using

# Prepare LlamaParse instance for PDF file that uses accurate mode which is true by default:
# it hallucinates so changing prompt

# pdf_parser = LlamaParse(result_type='text', verbose=True, continuous_mode=True, language= 'en',
#                     parsing_instruction = "You are parsing a regulatory document related to the Sri Lankan Tea Industry. Please give me a json with the title and date of issuance of the document"
#                     )  # "markdown" and "text" are available


# also doesn't work, don't use parsing prompt
pdf_parser = LlamaParse(result_type='markdown', num_workers=8)  # "markdown" and "text" are available

# Prepare LlamaParse instance for text file that uses fast mode:
text_parser = LlamaParse(result_type='markdown', Set_fast_mode = True, num_workers=8)  # "markdown" and "text" are available

# set workers =8 makes it 'fake' parallel

# default is text, modified to markdown for next time. Also, changed number of workers to 8 for next time


In [ ]:
# [del]
# # apparently this is synchronous batch parsing, extremely inefficient
# import pandas as pd
# from llama_parse import LlamaParse
# from llama_index.core import SimpleDirectoryReader


# # This function will handle processing each document based on its type
# def process_documents(row):
#     documents = []  # Initialize documents list locally for each row
#     file_type = row['PDF_or_text']  # Column that indicates if it's 'PDF' or 'text'

#     # If the file type is 'PDF', process with the parser
#     if file_type == 'PDF':
#         file_path = row['path']  # Assuming each row has a 'path' column
#         file_extractor = {".pdf": pdf_parser}
#         pdf_documents = SimpleDirectoryReader(input_files=[file_path], file_extractor=file_extractor).load_data()

#         # Append the processed PDF data to the documents list
#         # documents.extend(pdf_documents)
#         return pdf_documents
#     # If the file type is 'Text', process the text file
#     elif file_type == 'Text':
#         file_path = row['text_path']  # Assuming each row has a 'text_path' column
#         file_extractor = {".txt": text_parser}
#         text_documents = SimpleDirectoryReader(input_files=[file_path], file_extractor=file_extractor).load_data()
#         # @TODO document = Document(
#         # text="text",
#         # metadata={"filename": "<doc_file_name>", "category": "<category>"},
#         # )
#         # Append the processed text data to the documents list
#         # documents.extend(text_documents)
#         return text_documents
#     # If the file type is neither 'PDF' nor 'Text', return an empty list
#     else:
#       return None  # Return the processed documents for the row

# # Apply the process_documents function to each row of the DataFrame
# all_data['processed_data'] = all_data.apply(process_documents, axis=1)



In [ ]:
# # works perfectly but no metadata, opened an issue here: https://github.com/run-llama/llama_parse/issues/493
# import pandas as pd
# from llama_parse import LlamaParse
# import asyncio

# # Initialize parsers
# pdf_parser = LlamaParse(result_type='markdown', num_workers=8)
# text_parser = LlamaParse(result_type='markdown', Set_fast_mode=True, num_workers=8)


# # Async function to process PDF documents
# async def process_pdf_documents(pdf_paths):
#     if pdf_paths:  # Only process if there are PDF paths
#         documents = await pdf_parser.aload_data(pdf_paths)
#         return documents
#     return []

# # Async function to process Text documents
# async def process_text_documents(text_paths):
#     if text_paths:  # Only process if there are text paths
#         documents = await text_parser.aload_data(text_paths)
#         return documents
#     return []

# # Async function to process the DataFrame and save to CSV
# async def process_and_save_df(all_data):
#     # Collect file paths for PDFs and Text documents
#     pdf_paths = all_data[all_data['PDF_or_text'] == 'PDF']['path'].tolist()
#     text_paths = all_data[all_data['PDF_or_text'] == 'Text']['text_path'].tolist()

#     # Process PDFs and Texts concurrently
#     pdf_documents = await process_pdf_documents(pdf_paths)
#     text_documents = await process_text_documents(text_paths)

#     # Combine the results
#     all_documents = pdf_documents + text_documents

#     if all_documents:
#         return all_documents
#     else:
#         print("No documents found for processing.")
#         return []

# # Example usage
# async def main():
#     # Assuming `all_data` is a DataFrame that includes columns like 'PDF_or_text', 'path', 'text_path'
#     # all_data = pd.DataFrame({
#     #     'PDF_or_text': ['PDF', 'Text', 'PDF', 'Text'],
#     #     'path': ['./data/file1.pdf', '', './data/file3.pdf', ''],
#     #     'text_path': ['', './data/file2.txt', '', './data/file4.txt']
#     # })

#     # Process documents and get the list of processed documents
#     processed_documents = await process_and_save_df(all_data)

#     if processed_documents:
#         return processed_documents
#     else:
#         print("No documents to process.")

# # Run the asynchronous main function
# processed_documents = asyncio.run(main())


Parsing files: 100%|██████████| 29/29 [00:42<00:00,  1.48s/it]


In [ ]:
import pandas as pd
from llama_parse import LlamaParse
import asyncio

# Initialize parsers
pdf_parser = LlamaParse(result_type='markdown', num_workers=8)
text_parser = LlamaParse(result_type='markdown', Set_fast_mode=True, num_workers=8)


# Async function to process PDF documents
async def process_pdf_documents(pdf_paths):
    if pdf_paths:  # Only process if there are PDF paths
        file_extractor = {".pdf": pdf_parser}
        pdf_documents = SimpleDirectoryReader(input_files=pdf_paths, file_extractor=file_extractor).load_data()

        return pdf_documents
    return []

# Async function to process Text documents
async def process_text_documents(text_paths):
    if text_paths:  # Only process if there are text paths
        file_extractor = {".txt": text_parser}
        text_documents = SimpleDirectoryReader(input_files=text_paths, file_extractor=file_extractor).load_data()
        return text_documents
    return []

# Async function to process the DataFrame and save to CSV
async def process_and_save_df(all_data):
    # Collect file paths for PDFs and Text documents
    pdf_paths = all_data[all_data['PDF_or_text'] == 'PDF']['path'].tolist()
    text_paths = all_data[all_data['PDF_or_text'] == 'Text']['text_path'].tolist()

    # Process PDFs and Texts concurrently
    pdf_documents = await process_pdf_documents(pdf_paths)
    text_documents = await process_text_documents(text_paths)

    # Combine the results
    all_documents = pdf_documents + text_documents

    if all_documents:
        return all_documents
    else:
        print("No documents found for processing.")
        return []

# Example usage
async def main():
    # Assuming `all_data` is a DataFrame that includes columns like 'PDF_or_text', 'path', 'text_path'
    # all_data = pd.DataFrame({
    #     'PDF_or_text': ['PDF', 'Text', 'PDF', 'Text'],
    #     'path': ['./data/file1.pdf', '', './data/file3.pdf', ''],
    #     'text_path': ['', './data/file2.txt', '', './data/file4.txt']
    # })

    # Process documents and get the list of processed documents
    processed_documents = await process_and_save_df(all_data)

    if processed_documents:
        return processed_documents
    else:
        print("No documents to process.")

# Run the asynchronous main function
processed_documents = asyncio.run(main())


Started parsing the file under job_id 8d0003a2-989f-48fe-a9bb-5bf519673547
Started parsing the file under job_id 58a2e3a9-ed28-448f-89b1-e3273675c842
Started parsing the file under job_id e578ce21-8806-42cf-a9a1-dde7a951a9d3
Started parsing the file under job_id a76296cd-ee5a-463c-97aa-c3cd8f867667
Started parsing the file under job_id c861ae1d-db83-4275-abf1-eb5693fb7636
Started parsing the file under job_id 78e8be11-80ee-46ca-9145-fa4621953e28
Started parsing the file under job_id ea63608c-1289-410b-a799-3a12ae56becf
Started parsing the file under job_id 4b1e4b89-f6de-4627-b207-3395832822a7
Started parsing the file under job_id a7e0315a-c073-4f3d-b063-58491f5068db
Started parsing the file under job_id 1ceb5eeb-c50d-442d-b6dd-fd5e952d4ef0
Started parsing the file under job_id 5fac58c1-3d88-4c34-b8cd-cb05ac2def85
Started parsing the file under job_id fc58feda-9532-4b08-b5e5-694219a405f8
Started parsing the file under job_id dee46ec5-bc89-4db6-8b06-d6aebc2685d2
Started parsing the file 

In [ ]:
processed_documents[0]

Document(id_='a1060dd6-2f0b-4dc7-96c7-667f8130d706', embedding=None, metadata={'file_path': '/content/drive/MyDrive/Omdena_Challenge/new_LK_tea_dataset/Circulers/07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf', 'file_name': '07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf', 'file_type': 'application/pdf', 'file_size': 14710, 'creation_date': '2024-11-18', 'last_modified_date': '2024-11-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\n\n# GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA SAMPLES FOR TESTING\n\n# 1. Submission of tea sample\n\n- Sample of tea should be submitted to the Analytical Laboratory on any working day (Monday-Fri

Attributes to Extract:

id_: Unique identifier for the document.
embedding: An optional field, which may be None if not set.
metadata: A dictionary containing any associated metadata (in this case, it's empty).
relationships: A dictionary containing any relationships (empty in this case).
text: The content of the document itself.
mimetype: The MIME type of the document (in this case, it's 'text/plain').
start_char_idx, end_char_idx: These are None in your example, but may hold character index information if applicable.

Optional:

metadata_template, text_template, and other fields can be ignored if not needed.

In [ ]:
len(processed_documents)

625

In [ ]:
type(processed_documents)

list

In [ ]:
type(processed_documents[0])

llama_index.core.schema.Document

In [ ]:
import pandas as pd
import json
from llama_index.core.schema import Document

# Sample function to convert a single Document to a dictionary
def document_to_dict(document: Document):
    doc_dict = {
        "id": document.id_,
        "embedding": document.embedding,  # May be None
        "metadata": document.metadata,  # Empty in your case
        "excluded_embed_metadata_keys": document.excluded_embed_metadata_keys,
        "excluded_llm_metadata_keys": document.excluded_llm_metadata_keys,
        "relationships": document.relationships,  # Empty in your case
        "text": document.text,
        "mimetype": document.mimetype,
        "start_char_idx": document.start_char_idx,
        "end_char_idx": document.end_char_idx,
        "text_template": document.text_template,
        "metadata_template": document.metadata_template,
        "metadata_seperator": document.metadata_seperator
    }
    return doc_dict

# Convert the list of Document objects to a list of dictionaries
def documents_to_dicts(documents: list):
    return [document_to_dict(doc) for doc in documents]

# Convert the list of dictionaries to a pandas DataFrame
def documents_to_dataframe(documents: list):
    # Convert documents list to dicts
    doc_dicts = documents_to_dicts(documents)

    # Convert list of dictionaries to a DataFrame
    df = pd.DataFrame(doc_dicts)
    return df


# # Sample Document objects (this would normally come from your document processing)
# doc1 = Document(
#     id_='6af566d2-75f8-4bf1-b97e-a89c9e90066a',
#     embedding=None,
#     metadata={},
#     excluded_embed_metadata_keys=[],
#     excluded_llm_metadata_keys=[],
#     relationships={},
#     text="Sample text for document 1",
#     mimetype='text/plain',
#     start_char_idx=None,
#     end_char_idx=None,
#     text_template='{metadata_str}\n\n{content}',
#     metadata_template='{key}: {value}',
#     metadata_seperator='\n'
# )

# doc2 = Document(
#     id_='8bc31cd1-4c3e-44c9-b6b7-0fa56e987ac0',
#     embedding=None,
#     metadata={},
#     excluded_embed_metadata_keys=[],
#     excluded_llm_metadata_keys=[],
#     relationships={},
#     text="Sample text for document 2",
#     mimetype='text/plain',
#     start_char_idx=None,
#     end_char_idx=None,
#     text_template='{metadata_str}\n\n{content}',
#     metadata_template='{key}: {value}',
#     metadata_seperator='\n'
# )

# # Assuming we have a list of Document objects
# documents = [doc1, doc2]

# Convert to DataFrame
processed_df = documents_to_dataframe(processed_documents)

# # Show the resulting DataFrame
processed_df


,id,embedding,metadata,excluded_embed_metadata_keys,excluded_llm_metadata_keys,relationships,text,mimetype,start_char_idx,end_char_idx,text_template,metadata_template,metadata_seperator
0,a1060dd6-2f0b-4dc7-96c7-667f8130d706,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
1,3ab1ff71-ad31-4f45-a673-d8272f3876c4,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
2,f34dac8e-599d-4ef5-befb-c0c9bc51f0dc,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},NO_CONTENT_HERE,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
3,d4492d11-4e1d-4e54-970a-a05d46a7feb2,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
4,830e51b3-e6bb-4239-b478-3fda0ff27adb,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,7fa6f91c-772a-4594-aa17-6f92b8023027,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Definitions\n\n” Minister ” means the Minist...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
621,4e4ea0a1-e4c1-4ecd-acf8-b7933bf324eb,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
622,3dc6d2af-0509-4c7b-83da-36d862a2e037,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},remuneration as may be specified in the releva...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
623,7017bcbd-a2f3-4eab-bd30-e4521a5875fb,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n


In [ ]:
processed_df.to_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llama_processed_documents_markdown.csv',index=False)

# Extract metadata
this is done from the saved csv, and not all the chunked content we got from llamaparse via a vector database

This is discontinuation from the above code, it uses groq directly

models do have limits, so cannot feed whole parsed document e.g. markdown file. Will still need chunks, so first and last two (since the very last could be very small in size)

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Read the DataFrame (assuming the 6th column contains dictionaries with the 'text' key)
processed_df = pd.read_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llama_processed_documents_markdown.csv')


In [ ]:
import ast
# Convert the 'metadata' column from string representation to dictionary
processed_df['metadata'] = processed_df['metadata'].apply(ast.literal_eval)

# Now you can access dictionary keys
first_file_name = processed_df['metadata'].iloc[0]['file_name']
print(first_file_name)  # Output: file1.txt


07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf


In [ ]:
import pandas as pd

# Creating the DataFrame with 'metadata' column containing dictionaries
df_metadata = pd.DataFrame()
# Extracting the 'filename' from the 'metadata' column
df_metadata['file_name'] = processed_df['metadata'].apply(lambda x: x.get('file_name', None))
df_metadata['markdown_content'] = processed_df['text']

# Display the DataFrame
print(df_metadata)


                                             file_name  \
0    07_General-Instruction-for-the-Submission-of-T...   
1    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
2    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
3    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
4    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
..                                                 ...   
620                                         ACT_33.txt   
621                                         ACT_34.txt   
622                                         ACT_34.txt   
623                                         ACT_35.txt   
624                                         ACT_35.txt   

                                      markdown_content  
0    # ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...  
1    # MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...  
2                                      NO_CONTENT_HERE  
3    # Sri Lanka Tea Board Minimum Quality Standard...  
4    # Sri Lanka T

## Save parsed text to markdown files in main csv and also in folder because the parsed results have many other things



# only pick smaller mardown techunks, first and last two in a separate dataframe

In [ ]:
import pandas as pd


# Group by 'file_name' and apply a custom function to each group
def merge_markdown_content(group):

    # combined_text_all = first_row['markdown_content'] + " " + " ".join(last_two_rows['markdown_content'])
    # df.groupby(['file_name', 'file_path'], as_index=False)['markdown_content'].agg(lambda x: ' '.join(x))
    combined_markdown_all = group['markdown_content'].agg(lambda x:''.join(x))


    # Take the first row
    first_row = group.iloc[0]
    # Take the last two rows (if there are at least two)
    last_two_rows = group.tail(2)

    # Combine the first row text_content with the last two rows text_content
    combined_markdown_first_last = first_row['markdown_content'] + " " + " ".join(last_two_rows['markdown_content'])


    # Return a new series with file_name, file_path and the combined text_content
    return pd.Series({
        'file_name': first_row['file_name'],
        # 'file_path': first_row['file_path'],
        'markdown_content_long': combined_markdown_all,
        'markdown_content_short': combined_markdown_first_last

    })

# Apply the function to each group
merged_df = df_metadata.groupby('file_name', as_index=False).apply(merge_markdown_content)

# Drop the multi-index that `apply` creates
merged_df = merged_df.reset_index(drop=True)

# View the merged DataFrame
print(merged_df)


<ipython-input-8-2697a585077a>:9: FutureWarning: using <function merge_markdown_content.<locals>.<lambda> at 0x7a0175cf6710> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  combined_markdown_all = group['markdown_content'].agg(lambda x:''.join(x))
<ipython-input-8-2697a585077a>:9: FutureWarning: using <function merge_markdown_content.<locals>.<lambda> at 0x7a0175cf5c60> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  combined_markdown_all = group['markdown_content'].agg(lambda x:''.join(x))
<ipython-input-8-2697a585077a>:9: FutureWarning: using <function merge_markdown_content.<locals>.<lambda> at 0x7a0175cf5d80> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  combined_markdown_all = group['markdown_content'].agg(lambda x:''.join(x))
<ipython-input-8-2697a585077a>:9: FutureWarning: using <function merge_markdown

                                             file_name  \
0    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
1    07_General-Instruction-for-the-Submission-of-T...   
2                                       1998 10 30.pdf   
3                                        2158-03_E.pdf   
4                                        2245-29_E.pdf   
..                                                 ...   
163                           al-mqs-08 2008 04 24.pdf   
164  guidelines_on_covid-19_in_tea_plantations_eng_...   
165                                      mf bl 119.pdf   
166                                       scan0002.pdf   
167             tc - e - vi - 68 -vol 2 2003 03 07.pdf   

                                 markdown_content_long  \
0    1    # MINISTRY OF PLANTATION\n\n# SRI LANKA T...   
1    0    # ANALYTICAL LABORATORY OF SRILANKA TEA B...   
2    331    # Sri Lanka Tea Board\n\n574, Galle Roa...   
3    309    # PART I : SEC. (I) - GAZETTE EXTRAORDI...   
4    308    #

<ipython-input-8-2697a585077a>:9: FutureWarning: using <function merge_markdown_content.<locals>.<lambda> at 0x7a0175cf4dc0> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  combined_markdown_all = group['markdown_content'].agg(lambda x:''.join(x))
<ipython-input-8-2697a585077a>:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = df_metadata.groupby('file_name', as_index=False).apply(merge_markdown_content)


In [ ]:
import os
import getpass
# os.environ['LLAMA_CLOUD_API_KEY'] = getpass.getpass('Enter your LLamacloud API Key: ')


os.environ['GROQ_API_KEY'] = getpass.getpass('Enter your GROQ API Key: ')
groq_api=os.environ['GROQ_API_KEY']

Enter your GROQ API Key: ··········


In [ ]:
import pandas as pd
import json
from groq import Groq
from google.colab import userdata
import os
import re

# Assuming Groq API key is set up properly


client = Groq(api_key=groq_api)

def clean_text_for_metadata_extraction(text):
    """
    Clean and preprocess the extracted text content to make it suitable for metadata extraction.
    This will remove unwanted text such as 'Copyright' and any excessive spaces or line breaks.
    """
    if not isinstance(text, str):
        return None


    # Remove specific unwanted text like 'Copyright'
    # Remove specific unwanted text like 'Copyright' (case-insensitive) and everything after it
    text = re.sub(r'copyright[\s\S]*?(\n|\r|\Z)', '', text, flags=re.IGNORECASE)  # Remove copyright section and text after it

    # # Clean up the text by replacing line breaks and excessive spaces
    # text = re.sub(r'\n+', ' ', text)  # Replace multiple line breaks with a single space
    # text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    # text = text.strip()  # Remove leading/trailing spaces

    # # Optionally, trim to the first 500 characters for better performance
    # return text[:500]
    return text

def extract_metadata(row):
    """
    Extract title and published date from the document's text using the Groq API.
    The prompt is designed to instruct the Groq model to return a JSON object containing these two pieces of information.
    """
    text =  row['markdown_content_short']

    # Clean and preprocess the text before passing to the Groq API
    cleaned_text = clean_text_for_metadata_extraction(text)
    if not cleaned_text:
        return None

    prompt = (
        "Please process the following document and extract the relevant details in English in JSON format:"
        "- **parsed_title**: Provide the main document title,; if not, indicate 'None'."
        # "- **Laws/Acts/Amendments**: Identify any laws, acts, or amendments mentioned in the document, including their date.\n"
        # "- **Topics**: List the key topics discussed in the document.\n"
        "- **parsed_issue_date**: Provide the document's issuance date in DD.MM.YYYY format; if not, indicate 'None'."
        "- **parsed_reference_number**: If available, provide the reference number; if not, indicate 'None'."
        "Document text:\n"
        f"{cleaned_text}\n"
    )



    # Send the prompt to the Groq API
    try:
        completion = client.chat.completions.create(
            model="llama-3.2-1b-preview",  # Make sure this model is appropriate for your task
            messages=[
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": "```json"}
            ],
            stop="```",
        )

        # Try to extract the JSON response
        return completion.choices[0].message.content.strip("```json").strip()

    except (AttributeError, IndexError) as e:
        print(f"Error processing row: {e}")
        return None

# Apply the metadata extraction to the DataFrame
merged_df['metadata_returned'] = merged_df.apply(extract_metadata, axis = 1)


merged_df

In [ ]:
bu_merged_df = merged_df.copy()

In [ ]:
merged_df = bu_merged_df.copy()

In [ ]:
merged_df.head()

In [ ]:
# Using iloc to get the first row's metadata column value
first_metadata_value = merged_df.iloc[0]['metadata_returned']
first_metadata_value


In [ ]:
import pandas as pd
import json
import re


# Function to clean and parse the metadata string
def clean_json_string(json_string):
    try:
        # Check if the string is empty or contains invalid data
        if not json_string or json_string.isspace():
            print("Empty string or whitespace found")
            return None

        # Remove unwanted characters (non-ASCII, newline, etc.)
        cleaned_string = re.sub(r'[^\x00-\x7F]+', ' ', json_string)  # Remove non-ASCII characters
        cleaned_string = re.sub(r'[^a-zA-Z0-9.,;:?!()"\'\s/-]', '', cleaned_string)  # Remove non-English characters
        cleaned_string = cleaned_string.replace('<|end_header_id|>', '')  # Remove any specific unwanted tags
        cleaned_string = cleaned_string.replace('\n', ' ').replace('\r', ' ').strip()  # Remove newlines and extra spaces

        # Check for multiple JSON objects or broken JSON (simple check)
        if cleaned_string.count('{') > 1:
            # If there are multiple objects, extract the first valid one
            start = cleaned_string.find('{')
            end = cleaned_string.rfind('}') + 1
            cleaned_string = cleaned_string[start:end]

        # Try to load the cleaned string as JSON
        json_object = json.loads(cleaned_string)  # Try parsing the cleaned string as JSON

        return json_object
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None  # Return None if JSON is malformed
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None  # Handle any other unexpected errors

# Apply the cleaning function to the 'metadata_returned' column
merged_df['metadata_returned'] = merged_df['metadata_returned'].apply(clean_json_string)

# Expand the 'metadata_returned' column into individual columns in place
merged_df = pd.concat([merged_df, merged_df['metadata_returned'].apply(pd.Series)], axis=1)

# Drop the original 'metadata_returned' column to avoid duplication
merged_df.drop(columns=['metadata_returned'], inplace=True)

# Display the expanded DataFrame
print(merged_df)


In [ ]:
# works but misses some rows
# import pandas as pd
# import json
# import re



# # Function to clean and parse the metadata string
# def clean_json_string(json_string):
#     try:
#         # Check if the string is empty or contains invalid data
#         if not json_string or json_string.isspace():
#             print("Empty string or whitespace found")
#             return None

#         # Remove unwanted characters (non-ASCII, newline, etc.)
#         cleaned_string = re.sub(r'[^a-zA-Z0-9.,;:?!()"\'\s/-]', '', json_string)  # Remove non-English characters
#         cleaned_string = re.sub(r'[^\x00-\x7F]+', ' ', json_string)  # Remove non-ASCII characters
#         cleaned_string = cleaned_string.replace('<|end_header_id|>', '')  # Remove any specific unwanted tags
#         cleaned_string = cleaned_string.replace('\n', ' ').replace('\r', ' ').strip()  # Remove newlines and extra spaces

#         # Check for multiple JSON objects or broken JSON (simple check)
#         if cleaned_string.count('{') > 1:
#             # If there are multiple objects, extract the first valid one
#             start = cleaned_string.find('{')
#             end = cleaned_string.rfind('}') + 1
#             cleaned_string = cleaned_string[start:end]

#         # Try to load the cleaned string as JSON
#         json_object = json.loads(cleaned_string)  # Try parsing the cleaned string as JSON

#         return json_object
#     except json.JSONDecodeError as e:
#         print(f"Error decoding JSON: {e}")
#         return None  # Return None if JSON is malformed
#     except Exception as e:
#         print(f"Unexpected error: {e}")
#         return None  # Handle any other unexpected errors

# # Apply the cleaning function to the 'metadata_returned' column
# merged_df['metadata_returned'] = merged_df['metadata_returned'].apply(clean_json_string)

# # Remove rows where parsing failed (i.e., cleaned string is None)
# merged_df = merged_df.dropna(subset=['metadata_returned'])


# # Expanding the 'metadata_returned' column into individual columns in place
# merged_df = pd.concat([merged_df, merged_df['metadata_returned'].apply(pd.Series)], axis=1)

# # Drop the original 'metadata_returned' column to avoid duplication
# merged_df.drop(columns=['metadata_returned', 'title','issue_date','reference_number'], inplace=True)

# # Display the expanded DataFrame
# merged_df.head()




In [ ]:
# Optional: Save the updated DataFrame with the new columns to a new CSV file
merged_df.to_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llama_parsed_metadata.csv', index=False)


# By now I have a parsed result json (keep) and a metadata df, no need to save it
# append metadata results to main csv

In [ ]:
# prompt: append merged_df column based on filename to all_data and add columns metadata in full by extracting key values from dictionary and appending as columns and markdown_content_full

# Assuming 'all_data' is your main DataFrame and 'merged_df' is the DataFrame you created with the merged text content.
# Also assuming 'merged_df' has columns 'file_name', 'markdown_content_long', 'markdown_content_short'

def append_merged_df_to_all_data(all_data, merged_df):
  """
  Appends the merged_df columns based on the 'file_name' to the all_data DataFrame.
  """
  # all_data = all_data.merge(merged_df[['file_name', 'markdown_content_long', 'parsed_title','parsed_issue_date', 'parsed_reference_number']], on='file_name', how='left')
  all_data = all_data.merge(merged_df[['file_name', 'markdown_content_long', 'Title','Issue Date', 'Reference Number']], on='file_name', how='left')

  return all_data



# 1. Append merged_df columns to all_data
all_data = append_merged_df_to_all_data(all_data, merged_df)

# Print to confirm
print(all_data.head())

# get rows with no content here, and mark as bad quality in original csv, and remove for next step in next code

In [ ]:
# prompt: get markdown content rows where only NO CONTENT HERE exists and remove the rows

def remove_no_content_rows(df):
  """Removes rows from a DataFrame where 'markdown_content_short' contains only 'NO CONTENT HERE'."""
  df = df[~df['markdown_content_short'].str.contains(r'^NO_CONTENT_HERE$', na=False)]
  return df

# Assuming 'merged_df' is the DataFrame with the 'markdown_content_short' column
merged_df = remove_no_content_rows(merged_df)

# Now 'merged_df' has rows with only 'NO CONTENT HERE' removed.

# save main csv

In [ ]:
# Optional: Save the updated DataFrame with the new columns to a new CSV file
merged_df.to_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/v2_new_LK_tea_dataset_updated.csv', index=False)

# estimating number of token from markdown file

In [ ]:

# each chunk will be below 3,000? 30,000? via llamaparse i guess
import pandas as pd

# Function to count words and characters in text
def count_words_and_characters(text):
    words = text.split()  # Split by whitespace to get words
    num_words = len(words)
    num_characters = len(text)
    return num_words, num_characters

# Function to count words and characters for the 'text_content' column in the DataFrame
def count_stats_in_dataframe(df):
    total_words = 0
    total_characters = 0

    # Iterate over the 'text_content' column in the DataFrame
    for text in df['markdown_content_long'].dropna():  # Drop any NaN values in text_content
        num_words, num_characters = count_words_and_characters(text)
        total_words += num_words
        total_characters += num_characters

    return total_words, total_characters

# Main script
if __name__ == "__main__":

    # Count words and characters
    total_words, total_characters = count_stats_in_dataframe(all_data)

    # Output the results
    print("\nTotal Stats for 'text_content' column in DataFrame:")
    print(f"Total Words: {total_words}")
    print(f"Total Characters: {total_characters}")
